In [ ]:
%tensorflow_version 1.x
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding,Bidirectional,BatchNormalization
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical

from keras.callbacks import EarlyStopping
from keras.utils import  to_categorical
%matplotlib inline

In [ ]:
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer() 
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')


In [ ]:
stopwords = nltk.corpus.stopwords.words('english')

In [ ]:
def clean_text(text):
    
    text=re.sub(r"[^a-zA-Z]"," ",text)
    text=text.lower()
    text=re.sub(r"i'm","i am",text)
    text=re.sub(r"he's","he is",text)
    text=re.sub(r"she's","she is",text)
    text=re.sub(r"that's","that is",text)
    text=re.sub(r"what's","what is",text)
    text=re.sub(r"where's","where is",text)
    text=re.sub(r"\'ll"," will",text)
    text=re.sub(r"\'ve"," have",text)
    text=re.sub(r"\'re"," are",text)
    text=re.sub(r"\'d"," would",text)
    text=re.sub(r"won't","will not",text)
    text=re.sub(r"can't","cannot",text)
    text=re.sub(r"[-()\"#/@;:<>{}+=~|.?,]","",text)
    word_tokens = word_tokenize(text)
    filtered_sentence = [lemmatizer.lemmatize(w) for w in word_tokens if w not in stopwords]
    text=" ".join(filtered_sentence)
    import nltk
    nltk.download('words')
    words = set(nltk.corpus.words.words())
    n = " ".join(w for w in nltk.wordpunct_tokenize(text) \

            if w.lower() not in words or not w.isalpha())
    print(n)


    return text

In [ ]:
url='https://raw.githubusercontent.com/Prrriyanka/automatedessaygrading/master/actual%20dataset.csv'
#nurl = "https://raw.githubusercontent.com/Nehaaamishra/dv/master/same%20-%20Sheet1.csv"
turl = "https://raw.githubusercontent.com/Nehaaamishra/dv/master/unseen.csv"
train = pd.read_csv(url)



In [ ]:
train.head()

In [ ]:
train['avg_score']=np.round((train.score_1+train.score_2+train.score_3)/3)

In [ ]:
train2=train.drop(columns=['ID','Essayset','min_score','max_score','clarity','coherent'])

In [ ]:
train2.drop(columns=['score_1','score_2','score_3','score_4','score_5'],inplace=True)

In [ ]:

train.head(23000)

In [ ]:
train2.head()

In [ ]:
max_words = 2500
max_len = 50
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(train2.EssayText)
sequences = tok.texts_to_sequences(train2.EssayText)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
Y1=train2.avg_score

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(sequences_matrix,to_categorical(Y1),test_size=0.20)


In [ ]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = Bidirectional(LSTM(64))(layer)
    layer = Dense(64,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(64,name='FC2')(layer)
    #layer=BatchNormalization()(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(4,name='out_layer')(layer)
    layer = Activation('softmax')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [ ]:
model = RNN()
model.compile(loss='categorical_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])
model.summary()

model.fit(X_train,Y_train,batch_size=64,epochs=2,validation_data=(X_test,Y_test))


In [ ]:
y_pred_train=model.predict(sequences_matrix)

In [ ]:
tt1=pd.DataFrame(y_pred_train,columns=['out1','out2','out3','out4'])

In [ ]:
train3=train.drop(columns=['ID','min_score','max_score','EssayText'])

In [ ]:
train4=pd.concat([train3,tt1],axis=1)

In [ ]:
train4.head()

In [ ]:
!pip install category_encoders
import category_encoders as ce


In [ ]:
ce1=ce.TargetEncoder(cols = ['Essayset','clarity','coherent'], min_samples_leaf = 20)

In [ ]:
train4.loc[:,['Essayset','clarity','coherent']]=ce1.fit_transform(train4.loc[:,['Essayset','clarity','coherent']],train4.loc[:,['avg_score']])

In [ ]:
train4.head()


In [ ]:
X1=train4.drop(columns=['avg_score','clarity','coherent','score_1','score_2','score_3','score_4','score_5'])
Y1=train4.avg_score

In [ ]:
from xgboost import XGBClassifier
xgb=XGBClassifier(colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.01,
                 max_depth=4,
              
                 min_child_weight=0.5,
                 n_estimators=10000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42) 

In [ ]:
X1_train,X1_test,Y1_train,Y1_test=train_test_split(X1,Y1,test_size=0.2,random_state=123)

In [ ]:
xgb.fit(X1_train,Y1_train)

In [ ]:
yy_test=xgb.predict(X1_test)

In [ ]:
from sklearn.metrics import  accuracy_score,classification_report

In [ ]:
accuracy_score(Y1_test,yy_test)

In [ ]:
print(classification_report(Y1_test,yy_test))

In [ ]:
from google.colab import files
nurl = files.upload()

In [ ]:
import io

test=pd.read_csv(io.BytesIO(nurl['essays.csv']))
testprint=pd.read_csv(io.BytesIO(nurl['essays.csv']))


In [ ]:
test2=test.drop(columns=['ID'])

In [ ]:
test2.tail()


In [ ]:
test2['EssayText']=test2['EssayText'].apply(clean_text)

In [ ]:
sequences_test = tok.texts_to_sequences(test2.EssayText)
sequences_matrix_test = sequence.pad_sequences(sequences_test,maxlen=max_len)


In [ ]:
test2_test=model.predict(sequences_matrix_test)

In [ ]:
testdf=pd.DataFrame(test2_test,columns=['out1','out2','out3','out4'])

In [ ]:
test3=pd.concat([test2,testdf],axis=1)

In [ ]:
test3.drop(columns=['EssayText'],inplace=True)
test3.head()

In [ ]:

test.head()


In [ ]:
#test3.drop(columns=['score_1','score_2','score_3','score_4','score_5'],inplace=True)

test3.head()


In [ ]:
final_out=xgb.predict(test3)

In [ ]:
out=pd.DataFrame()


In [ ]:
out['id']=test.ID
out['essay_set']=test.Essayset
out['your_essay']=testprint.EssayText


In [ ]:
out['essay_score']=final_out
out.head()

In [ ]:
out.to_csv('Out.csv',index=None)
nn=pd.read_csv('Out.csv')

In [ ]:
nn.head()

In [ ]:
#absolute_path = os.path.abspath(os.path.dirname('output.csv'))


In [ ]:
from google.colab import files
files.download("Out.csv")